In [333]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings(action = 'ignore')

In [334]:
# a = pd.read_csv('final_DE.csva',index_col=0)
# a

In [335]:
sample_df = pd.read_excel('FINAL_NUM_1029.xlsx')
sample_df.drop('STD_DAY',axis=1,inplace=True)

In [336]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   DONG_CODE            467 non-null    int64 
 1   SI_DO                467 non-null    object
 2   GU                   467 non-null    object
 3   DONG                 467 non-null    object
 4   ACADEMY_NUM          467 non-null    int64 
 5   KINDER_NUM           467 non-null    int64 
 6   FIRE_NUM             467 non-null    int64 
 7   ELE_SCH_NUM          467 non-null    int64 
 8   MID_SCH_NUM          467 non-null    int64 
 9   HIGH_SCH_NUM         467 non-null    int64 
 10  CCTV_NUM             467 non-null    int64 
 11  POLICE_NUM           467 non-null    int64 
 12  BIKE_NUM             467 non-null    int64 
 13  CAR_SHR_NUM          467 non-null    int64 
 14  BUS_NUM              467 non-null    int64 
 15  SUBWAY_NUM           467 non-null    int64 
 16  SAFE_DLV

In [337]:
change_df = pd.read_excel('행정구역분류.xlsx',header=1)
change_df.head(3)

,시도,시군구,행정구역명,행정동\n(행정기관명),법정동,행정구역분류\n(개정_8자리),행정구역분류\n(7자리),행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역\n분할여부,행정동 영문명칭,비고
0,서울특별시,서울특별시,서울특별시,서울특별시,서울특별시,11.0,11.0,1100000000,19880423,1100000000,NaN,Seoul,NaN
1,서울특별시,종로구,종로구,종로구,종로구,11010.0,11010.0,1111000000,19880423,1111000000,NaN,Jongno-gu,NaN
2,서울특별시,종로구,청운효자동,청운효자동,청운동,11010720.0,1101072.0,1111051500,20081101,1111010100,NaN,Cheongunhyoja-dong,NaN


In [338]:
seoul_df = change_df[change_df['시도']=='서울특별시']
seoul_df = seoul_df[seoul_df['행정구역명'].str[-1] =='동']
seoul_df = seoul_df[['시군구','행정구역명','법정동','법정동코드','행정기관코드','법정동 관할구역\n분할여부']]
seoul_df.columns = ['GU','DONG','법정동','DONG_CODE','행정기관코드','분할여부']
non_div = seoul_df[seoul_df['분할여부'].isnull()][['GU','DONG','DONG_CODE','행정기관코드']]
non_div

,GU,DONG,DONG_CODE,행정기관코드
2,종로구,청운효자동,1111010100,1111051500
3,종로구,청운효자동,1111010200,1111051500
4,종로구,청운효자동,1111010300,1111051500
5,종로구,청운효자동,1111010400,1111051500
6,종로구,청운효자동,1111010500,1111051500
...,...,...,...,...
726,송파구,오금동,1171011200,1171057000
729,송파구,석촌동,1171010500,1171060000
730,송파구,삼전동,1171010600,1171061000
748,강동구,강일동,1174011000,1174051500


In [339]:
remove_du = non_div.drop_duplicates(['행정기관코드'])
remove_du

,GU,DONG,DONG_CODE,행정기관코드
2,종로구,청운효자동,1111010100,1111051500
12,종로구,사직동,1111010600,1111053000
24,종로구,삼청동,1111013900,1111054000
31,종로구,부암동,1111018400,1111055000
34,종로구,평창동,1111018200,1111056000
...,...,...,...,...
726,송파구,오금동,1171011200,1171057000
729,송파구,석촌동,1171010500,1171060000
730,송파구,삼전동,1171010600,1171061000
748,강동구,강일동,1174011000,1174051500


In [340]:
con_df1 = pd.merge(sample_df,non_div)
con_df1

,DONG_CODE,SI_DO,GU,DONG,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,...,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,행정기관코드
0,1114016500,서울특별시,중구,황학동,3,20,0,0,0,0,...,6,3,1,0,16,1060,7,21,0,1114067000
1,1114017100,서울특별시,중구,중림동,9,0,0,0,0,1,...,1,0,0,0,10,1060,12,15,0,1114068000
2,1117010100,서울특별시,용산구,후암동,28,10,1,4,0,0,...,1,1,0,0,16,1625,5,14,0,1117051000
3,1117010500,서울특별시,용산구,남영동,1,0,0,0,0,0,...,5,1,0,0,2,1625,2,7,0,1117053000
4,1117011900,서울특별시,용산구,효창동,22,20,0,2,0,0,...,1,0,1,0,10,1625,2,8,1,1117058000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,1154510100,서울특별시,금천구,가산동,36,10,0,0,0,0,...,24,13,9,4,170,1876,19,212,0,1154551000
64,1129010100,서울특별시,성북구,성북동,9,0,0,2,2,2,...,2,1,0,0,14,1783,0,28,0,1129052500
65,1129013500,서울특별시,성북구,종암동,89,20,3,4,2,1,...,7,6,2,1,34,1783,19,35,0,1129070500
66,1129013900,서울특별시,성북구,석관동,47,50,0,4,1,1,...,6,2,0,0,22,1783,3,30,0,1129081000


In [341]:
non_div_result = con_df1.groupby('행정기관코드').sum()
non_div_result = non_div_result.drop(['DONG_CODE'],axis=1)
non_div_result

,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,...,SPORT_NUM,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM
행정기관코드,,,,,,,,,,,,,,,,,,,,,
1111053000,0,0,0,0,0,0,8,1,2,0,...,0,2,2,1,0,2,2016,0,10,1
1111054000,1,10,0,0,0,0,14,0,3,0,...,1,2,0,1,0,0,2016,0,19,1
1111055000,5,0,0,0,0,0,15,0,2,2,...,0,1,0,0,0,2,2016,2,13,1
1111056000,23,0,0,0,0,1,18,1,5,0,...,0,4,4,1,0,4,2016,7,29,0
1111057000,20,10,0,2,0,0,13,0,3,1,...,0,0,1,0,0,2,2016,3,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171057000,148,70,0,6,5,2,25,1,13,4,...,0,12,13,1,0,32,2099,5,28,1
1171060000,112,30,0,2,0,0,21,0,3,5,...,0,18,6,1,0,38,2099,15,39,0
1171061000,127,20,0,2,1,1,14,1,5,5,...,0,19,8,1,0,28,2099,11,30,0


In [342]:
non_div_result = pd.merge(non_div_result, remove_du,right_on='행정기관코드',left_index=True)
non_div_result

,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,...,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,GU,DONG,DONG_CODE,행정기관코드
12,0,0,0,0,0,0,8,1,2,0,...,0,2,2016,0,10,1,종로구,사직동,1111010600,1111053000
24,1,10,0,0,0,0,14,0,3,0,...,0,0,2016,0,19,1,종로구,삼청동,1111013900,1111054000
31,5,0,0,0,0,0,15,0,2,2,...,0,2,2016,2,13,1,종로구,부암동,1111018400,1111055000
34,23,0,0,0,0,1,18,1,5,0,...,0,4,2016,7,29,0,종로구,평창동,1111018200,1111056000
36,20,10,0,2,0,0,13,0,3,1,...,0,2,2016,3,10,0,종로구,무악동,1111018700,1111057000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,148,70,0,6,5,2,25,1,13,4,...,0,32,2099,5,28,1,송파구,오금동,1171011200,1171057000
729,112,30,0,2,0,0,21,0,3,5,...,0,38,2099,15,39,0,송파구,석촌동,1171010500,1171060000
730,127,20,0,2,1,1,14,1,5,5,...,0,28,2099,11,30,0,송파구,삼전동,1171010600,1171061000
748,80,70,0,6,6,1,16,1,15,10,...,0,32,4327,9,16,0,강동구,강일동,1174011000,1174051500


In [343]:
non_div_result.drop('DONG_CODE',inplace=True,axis=1)

In [344]:
non_div_result.rename(columns = {'행정기관코드':'DONG_CODE'},inplace=True)
non_div_result

,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,...,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,GU,DONG,DONG_CODE
12,0,0,0,0,0,0,8,1,2,0,...,1,0,2,2016,0,10,1,종로구,사직동,1111053000
24,1,10,0,0,0,0,14,0,3,0,...,1,0,0,2016,0,19,1,종로구,삼청동,1111054000
31,5,0,0,0,0,0,15,0,2,2,...,0,0,2,2016,2,13,1,종로구,부암동,1111055000
34,23,0,0,0,0,1,18,1,5,0,...,1,0,4,2016,7,29,0,종로구,평창동,1111056000
36,20,10,0,2,0,0,13,0,3,1,...,0,0,2,2016,3,10,0,종로구,무악동,1111057000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,148,70,0,6,5,2,25,1,13,4,...,1,0,32,2099,5,28,1,송파구,오금동,1171057000
729,112,30,0,2,0,0,21,0,3,5,...,1,0,38,2099,15,39,0,송파구,석촌동,1171060000
730,127,20,0,2,1,1,14,1,5,5,...,1,0,28,2099,11,30,0,송파구,삼전동,1171061000
748,80,70,0,6,6,1,16,1,15,10,...,1,0,32,4327,9,16,0,강동구,강일동,1174051500


In [270]:
# non_div_result.columns

In [271]:
# non_div_result.columns = ['ACADEMY_NUM', 'KINDER_NUM', 'FIRE_NUM', 'ELE_SCH_NUM', 'MID_SCH_NUM',
#        'HIGH_SCH_NUM', 'CCTV_NUM', 'POLICE_NUM', 'BIKE_NUM', 'CAR_SHR_NUM',
#        'BUS_NUM', 'SUBWAY_NUM', 'SAFE_DLVR_NUM', 'DPTM_NUM', 'ANI_HSPT_NUM',
#        'PHARM_NUM', 'LEISURE_NUM', 'KIDS_NUM', 'HOSPITAL_NUM', 'SPORT_NUM',
#        'GYM_NUM', 'GOLF_NUM', 'STARBUCKS_NUM', 'MC_NUM', 'CON_NUM',
#        'NOISE_VIBRATION_NUM', 'CHILD_MED_NUM', 'CAFE_NUM', 'PARK_NUM','GU', 'DONG', 'DONG_CODE']

In [272]:
non_div_result['DPTM_NUM'].describe()

count    68.000000
mean      0.044118
std       0.206883
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000
Name: DPTM_NUM, dtype: float64

In [345]:
div_df = seoul_df[seoul_df['분할여부']=='분할연계'][['GU','DONG','DONG_CODE','행정기관코드']]
div_df

,GU,DONG,DONG_CODE,행정기관코드
11,종로구,청운효자동,1111011900,1111051500
21,종로구,사직동,1111011900,1111053000
47,종로구,종로1.2.3.4가동,1111011900,1111061500
56,종로구,종로1.2.3.4가동,1111013000,1111061500
84,종로구,혜화동,1111013000,1111065000
...,...,...,...,...
761,강동구,성내1동,1174010800,1174064000
762,강동구,성내2동,1174010800,1174065000
763,강동구,성내3동,1174010800,1174066000
765,강동구,둔촌1동,1174010600,1174069000


In [346]:
div_dong_count = div_df.groupby('DONG_CODE').count()[['행정기관코드']]
div_dong_count.columns = ['DONG_COUNT']
div_dong_count

,DONG_COUNT
DONG_CODE,
1111011900,3
1111013000,2
1111017400,3
1111017500,2
1114011500,2
...,...
1174010300,2
1174010600,2
1174010700,3


In [347]:
con_df2 = pd.merge(sample_df, div_dong_count,left_on='DONG_CODE',right_index=True)
con_df2.set_index('DONG_CODE',inplace=True)

In [348]:
con_df2

,SI_DO,GU,DONG,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,...,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,DONG_COUNT
DONG_CODE,,,,,,,,,,,,,,,,,,,,,
1114014200,서울특별시,중구,예장동,0,0,0,4,0,1,13,...,1,0,0,0,2,1060,0,5,1,2
1114013200,서울특별시,중구,충무로4가,1,0,0,0,0,0,8,...,0,0,0,0,2,1060,4,14,0,2
1114013300,서울특별시,중구,충무로5가,0,0,0,0,0,0,10,...,0,0,1,0,6,1060,1,7,0,2
1114013600,서울특별시,중구,묵정동,5,0,0,0,0,0,14,...,0,0,0,0,4,1060,4,8,0,2
1114014400,서울특별시,중구,장충동2가,0,0,0,2,0,0,58,...,0,0,0,0,8,1060,0,18,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130510200,서울특별시,강북구,번동,65,30,3,6,2,0,104,...,7,6,1,2,52,2204,13,36,2,3
1130510300,서울특별시,강북구,수유동,116,30,1,8,5,1,198,...,16,4,0,0,102,2204,17,95,0,5
1132010700,서울특별시,도봉구,창동,205,130,1,18,3,3,105,...,31,12,2,0,94,1289,12,111,3,5


In [349]:
con_df2 = con_df2.iloc[:,3:]
con_df2

,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,...,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,DONG_COUNT
DONG_CODE,,,,,,,,,,,,,,,,,,,,,
1114014200,0,0,0,4,0,1,13,0,0,0,...,1,0,0,0,2,1060,0,5,1,2
1114013200,1,0,0,0,0,0,8,0,1,0,...,0,0,0,0,2,1060,4,14,0,2
1114013300,0,0,0,0,0,0,10,0,0,1,...,0,0,1,0,6,1060,1,7,0,2
1114013600,5,0,0,0,0,0,14,0,0,0,...,0,0,0,0,4,1060,4,8,0,2
1114014400,0,0,0,2,0,0,58,1,5,2,...,0,0,0,0,8,1060,0,18,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130510200,65,30,3,6,2,0,104,2,22,4,...,7,6,1,2,52,2204,13,36,2,3
1130510300,116,30,1,8,5,1,198,4,19,10,...,16,4,0,0,102,2204,17,95,0,5
1132010700,205,130,1,18,3,3,105,2,32,8,...,31,12,2,0,94,1289,12,111,3,5


In [350]:
cal_df = con_df2.div(con_df2['DONG_COUNT'], axis=0) 
cal_df

,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,...,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,DONG_COUNT
DONG_CODE,,,,,,,,,,,,,,,,,,,,,
1114014200,0.000000,0.0,0.0,2.0,0.000000,0.5,6.500000,0.000000,0.000000,0.000000,...,0.500000,0.0,0.000000,0.000000,1.000000,530.000000,0.000000,2.5,0.500000,1.0
1114013200,0.500000,0.0,0.0,0.0,0.000000,0.0,4.000000,0.000000,0.500000,0.000000,...,0.000000,0.0,0.000000,0.000000,1.000000,530.000000,2.000000,7.0,0.000000,1.0
1114013300,0.000000,0.0,0.0,0.0,0.000000,0.0,5.000000,0.000000,0.000000,0.500000,...,0.000000,0.0,0.500000,0.000000,3.000000,530.000000,0.500000,3.5,0.000000,1.0
1114013600,2.500000,0.0,0.0,0.0,0.000000,0.0,7.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,2.000000,530.000000,2.000000,4.0,0.000000,1.0
1114014400,0.000000,0.0,0.0,1.0,0.000000,0.0,29.000000,0.500000,2.500000,1.000000,...,0.000000,0.0,0.000000,0.000000,4.000000,530.000000,0.000000,9.0,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130510200,21.666667,10.0,1.0,2.0,0.666667,0.0,34.666667,0.666667,7.333333,1.333333,...,2.333333,2.0,0.333333,0.666667,17.333333,734.666667,4.333333,12.0,0.666667,1.0
1130510300,23.200000,6.0,0.2,1.6,1.000000,0.2,39.600000,0.800000,3.800000,2.000000,...,3.200000,0.8,0.000000,0.000000,20.400000,440.800000,3.400000,19.0,0.000000,1.0
1132010700,41.000000,26.0,0.2,3.6,0.600000,0.6,21.000000,0.400000,6.400000,1.600000,...,6.200000,2.4,0.400000,0.000000,18.800000,257.800000,2.400000,22.2,0.600000,1.0


In [351]:
div_result_df = pd.merge(div_df,cal_df,left_on='DONG_CODE',right_index=True)
div_result_df = div_result_df.drop(['DONG_CODE','DONG_COUNT'],axis=1)
div_result_df

,GU,DONG,행정기관코드,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,...,SPORT_NUM,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM
11,종로구,청운효자동,1111051500,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2.666667,...,0.0,0.0,0.000000,0.333333,0.0,0.000000,672.000000,0.333333,3.666667,0.666667
21,종로구,사직동,1111053000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2.666667,...,0.0,0.0,0.000000,0.333333,0.0,0.000000,672.000000,0.333333,3.666667,0.666667
47,종로구,종로1.2.3.4가동,1111061500,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2.666667,...,0.0,0.0,0.000000,0.333333,0.0,0.000000,672.000000,0.333333,3.666667,0.666667
56,종로구,종로1.2.3.4가동,1111061500,0.500000,0.000000,0.0,0.000000,0.000000,0.0,3.000000,...,0.0,0.0,0.000000,0.000000,0.0,1.000000,1008.000000,0.000000,4.500000,0.000000
84,종로구,혜화동,1111065000,0.500000,0.000000,0.0,0.000000,0.000000,0.0,3.000000,...,0.0,0.0,0.000000,0.000000,0.0,1.000000,1008.000000,0.000000,4.500000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,강동구,성내1동,1174064000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,...,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000
762,강동구,성내2동,1174065000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,...,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000
763,강동구,성내3동,1174066000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,...,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000
765,강동구,둔촌1동,1174069000,38.000000,0.000000,0.0,2.000000,1.500000,1.0,32.500000,...,0.0,4.0,3.000000,0.000000,0.5,17.000000,2163.500000,0.000000,17.500000,1.000000


In [352]:
div_result_df.rename(columns = {'행정기관코드':'DONG_CODE'},inplace=True)

In [353]:
div_result_df

,GU,DONG,DONG_CODE,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,...,SPORT_NUM,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM
11,종로구,청운효자동,1111051500,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2.666667,...,0.0,0.0,0.000000,0.333333,0.0,0.000000,672.000000,0.333333,3.666667,0.666667
21,종로구,사직동,1111053000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2.666667,...,0.0,0.0,0.000000,0.333333,0.0,0.000000,672.000000,0.333333,3.666667,0.666667
47,종로구,종로1.2.3.4가동,1111061500,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2.666667,...,0.0,0.0,0.000000,0.333333,0.0,0.000000,672.000000,0.333333,3.666667,0.666667
56,종로구,종로1.2.3.4가동,1111061500,0.500000,0.000000,0.0,0.000000,0.000000,0.0,3.000000,...,0.0,0.0,0.000000,0.000000,0.0,1.000000,1008.000000,0.000000,4.500000,0.000000
84,종로구,혜화동,1111065000,0.500000,0.000000,0.0,0.000000,0.000000,0.0,3.000000,...,0.0,0.0,0.000000,0.000000,0.0,1.000000,1008.000000,0.000000,4.500000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,강동구,성내1동,1174064000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,...,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000
762,강동구,성내2동,1174065000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,...,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000
763,강동구,성내3동,1174066000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,...,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000
765,강동구,둔촌1동,1174069000,38.000000,0.000000,0.0,2.000000,1.500000,1.0,32.500000,...,0.0,4.0,3.000000,0.000000,0.5,17.000000,2163.500000,0.000000,17.500000,1.000000


In [354]:
# div_result_df.columns = ['GU', 'DONG', 'DONG_CODE',   'ACADEMY_NUM', 'KINDER_NUM', 'FIRE_NUM',
#        'ELE_SCH_NUM', 'MID_SCH_NUM', 'HIGH_SCH_NUM', 'CCTV_NUM', 'POLICE_NUM',
#        'BIKE_NUM', 'CAR_SHR_NUM', 'BUS_NUM', 'SUBWAY_NUM', 'SAFE_DLVR_NUM',
#        'DPTM_NUM', 'ANI_HSPT_NUM', 'PHARM_NUM', 'LEISURE_NUM', 'KIDS_NUM',
#        'HOSPITAL_NUM', 'SPORT_NUM', 'GYM_NUM', 'GOLF_NUM', 'STARBUCKS_NUM',
#        'MC_NUM', 'CON_NUM', 'NOISE_VIBRATION_NUM', 'CHILD_MED_NUM', 'CAFE_NUM',
#        'PARK_NUM']

In [357]:
non_div_result.columns

Index(['ACADEMY_NUM', 'KINDER_NUM', 'FIRE_NUM', 'ELE_SCH_NUM', 'MID_SCH_NUM',
       'HIGH_SCH_NUM', 'CCTV_NUM', 'POLICE_NUM', 'BIKE_NUM', 'CAR_SHR_NUM',
       'BUS_NUM', 'SUBWAY_NUM', 'SAFE_DLVR_NUM', 'DPTM_NUM', 'ANI_HSPT_NUM',
       'PHARM_NUM', 'LEISURE_NUM', 'KIDS_NUM', 'HOSPITAL_NUM', 'SPORT_NUM',
       'GYM_NUM', 'GOLF_NUM', 'STARBUCKS_NUM', 'MC_NUM', 'CON_NUM',
       'NOISE_VIBRATION_NUM', 'CHILD_MED_NUM', 'CAFE_NUM', 'PARK_NUM', 'GU',
       'DONG', 'DONG_CODE'],
      dtype='object')

In [358]:
div_result_df.columns

Index(['GU', 'DONG', 'DONG_CODE', 'ACADEMY_NUM', 'KINDER_NUM', 'FIRE_NUM',
       'ELE_SCH_NUM', 'MID_SCH_NUM', 'HIGH_SCH_NUM', 'CCTV_NUM', 'POLICE_NUM',
       'BIKE_NUM', 'CAR_SHR_NUM', 'BUS_NUM', 'SUBWAY_NUM', 'SAFE_DLVR_NUM',
       'DPTM_NUM', 'ANI_HSPT_NUM', 'PHARM_NUM', 'LEISURE_NUM', 'KIDS_NUM',
       'HOSPITAL_NUM', 'SPORT_NUM', 'GYM_NUM', 'GOLF_NUM', 'STARBUCKS_NUM',
       'MC_NUM', 'CON_NUM', 'NOISE_VIBRATION_NUM', 'CHILD_MED_NUM', 'CAFE_NUM',
       'PARK_NUM'],
      dtype='object')

In [359]:
non_div_result = non_div_result[div_result_df.columns]
non_div_result

,GU,DONG,DONG_CODE,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,...,SPORT_NUM,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM
12,종로구,사직동,1111053000,0,0,0,0,0,0,8,...,0,2,2,1,0,2,2016,0,10,1
24,종로구,삼청동,1111054000,1,10,0,0,0,0,14,...,1,2,0,1,0,0,2016,0,19,1
31,종로구,부암동,1111055000,5,0,0,0,0,0,15,...,0,1,0,0,0,2,2016,2,13,1
34,종로구,평창동,1111056000,23,0,0,0,0,1,18,...,0,4,4,1,0,4,2016,7,29,0
36,종로구,무악동,1111057000,20,10,0,2,0,0,13,...,0,0,1,0,0,2,2016,3,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,송파구,오금동,1171057000,148,70,0,6,5,2,25,...,0,12,13,1,0,32,2099,5,28,1
729,송파구,석촌동,1171060000,112,30,0,2,0,0,21,...,0,18,6,1,0,38,2099,15,39,0
730,송파구,삼전동,1171061000,127,20,0,2,1,1,14,...,0,19,8,1,0,28,2099,11,30,0
748,강동구,강일동,1174051500,80,70,0,6,6,1,16,...,0,3,1,1,0,32,4327,9,16,0


In [360]:
result_df = pd.concat([non_div_result, div_result_df],axis=0)
result_df

,GU,DONG,DONG_CODE,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,...,SPORT_NUM,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM
12,종로구,사직동,1111053000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,8.0,...,0.0,2.0,2.000000,1.000000,0.0,2.000000,2016.000000,0.0,10.000000,1.0
24,종로구,삼청동,1111054000,1.000000,10.000000,0.0,0.000000,0.000000,0.0,14.0,...,1.0,2.0,0.000000,1.000000,0.0,0.000000,2016.000000,0.0,19.000000,1.0
31,종로구,부암동,1111055000,5.000000,0.000000,0.0,0.000000,0.000000,0.0,15.0,...,0.0,1.0,0.000000,0.000000,0.0,2.000000,2016.000000,2.0,13.000000,1.0
34,종로구,평창동,1111056000,23.000000,0.000000,0.0,0.000000,0.000000,1.0,18.0,...,0.0,4.0,4.000000,1.000000,0.0,4.000000,2016.000000,7.0,29.000000,0.0
36,종로구,무악동,1111057000,20.000000,10.000000,0.0,2.000000,0.000000,0.0,13.0,...,0.0,0.0,1.000000,0.000000,0.0,2.000000,2016.000000,3.0,10.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,강동구,성내1동,1174064000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.0,...,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.0,47.666667,0.0
762,강동구,성내2동,1174065000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.0,...,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.0,47.666667,0.0
763,강동구,성내3동,1174066000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.0,...,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.0,47.666667,0.0
765,강동구,둔촌1동,1174069000,38.000000,0.000000,0.0,2.000000,1.500000,1.0,32.5,...,0.0,4.0,3.000000,0.000000,0.5,17.000000,2163.500000,0.0,17.500000,1.0


In [361]:
len(result_df['DONG_CODE'].unique())

408

In [363]:
remove_result_du =result_df.drop_duplicates(['DONG_CODE'])[['GU','DONG','DONG_CODE']]
remove_result_du

,GU,DONG,DONG_CODE
12,종로구,사직동,1111053000
24,종로구,삼청동,1111054000
31,종로구,부암동,1111055000
34,종로구,평창동,1111056000
36,종로구,무악동,1111057000
...,...,...,...
761,강동구,성내1동,1174064000
762,강동구,성내2동,1174065000
763,강동구,성내3동,1174066000
765,강동구,둔촌1동,1174069000


In [253]:
result_sum = result_df.groupby('DONG_CODE').sum()
result_sum


,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,...,SPORT_NUM,GYM_NUM,GOLF_NUM,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM
DONG_CODE,,,,,,,,,,,,,,,,,,,,,
1111051500,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2.666667,0.000000,1.666667,1.000000,...,0.0,0.0,0.000000,0.333333,0.0,0.000000,672.000000,0.333333,3.666667,0.666667
1111053000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,10.666667,1.000000,3.666667,1.000000,...,0.0,2.0,2.000000,1.333333,0.0,2.000000,2688.000000,0.333333,13.666667,1.666667
1111054000,1.000000,10.000000,0.0,0.000000,0.000000,0.0,14.000000,0.000000,3.000000,0.000000,...,1.0,2.0,0.000000,1.000000,0.0,0.000000,2016.000000,0.000000,19.000000,1.000000
1111055000,5.000000,0.000000,0.0,0.000000,0.000000,0.0,15.000000,0.000000,2.000000,2.000000,...,0.0,1.0,0.000000,0.000000,0.0,2.000000,2016.000000,2.000000,13.000000,1.000000
1111056000,23.000000,0.000000,0.0,0.000000,0.000000,1.0,18.000000,1.000000,5.000000,0.000000,...,0.0,4.0,4.000000,1.000000,0.0,4.000000,2016.000000,7.000000,29.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174065000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,...,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000
1174066000,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,...,0.0,10.0,4.666667,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000
1174068500,100.000000,20.000000,1.0,6.000000,1.000000,0.0,119.000000,0.000000,9.000000,10.000000,...,0.0,18.0,9.000000,2.000000,0.0,76.000000,4327.000000,10.000000,63.000000,1.000000


In [254]:

result_df2 = pd.merge(result_sum, remove_result_du,right_on='DONG_CODE',left_index=True)
result_df2

,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,...,STARBUCKS_NUM,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,GU,DONG,DONG_CODE
11,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2.666667,0.000000,1.666667,1.000000,...,0.333333,0.0,0.000000,672.000000,0.333333,3.666667,0.666667,종로구,청운효자동,1111051500
12,0.000000,0.000000,0.0,0.000000,0.000000,0.0,10.666667,1.000000,3.666667,1.000000,...,1.333333,0.0,2.000000,2688.000000,0.333333,13.666667,1.666667,종로구,사직동,1111053000
24,1.000000,10.000000,0.0,0.000000,0.000000,0.0,14.000000,0.000000,3.000000,0.000000,...,1.000000,0.0,0.000000,2016.000000,0.000000,19.000000,1.000000,종로구,삼청동,1111054000
31,5.000000,0.000000,0.0,0.000000,0.000000,0.0,15.000000,0.000000,2.000000,2.000000,...,0.000000,0.0,2.000000,2016.000000,2.000000,13.000000,1.000000,종로구,부암동,1111055000
34,23.000000,0.000000,0.0,0.000000,0.000000,1.0,18.000000,1.000000,5.000000,0.000000,...,1.000000,0.0,4.000000,2016.000000,7.000000,29.000000,0.000000,종로구,평창동,1111056000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,...,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000,강동구,성내2동,1174065000
763,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,...,1.333333,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000,강동구,성내3동,1174066000
764,100.000000,20.000000,1.0,6.000000,1.000000,0.0,119.000000,0.000000,9.000000,10.000000,...,2.000000,0.0,76.000000,4327.000000,10.000000,63.000000,1.000000,강동구,길동,1174068500
765,38.000000,0.000000,0.0,2.000000,1.500000,1.0,32.500000,1.000000,5.500000,2.000000,...,0.000000,0.5,17.000000,2163.500000,0.000000,17.500000,1.000000,강동구,둔촌1동,1174069000


In [129]:
result_df2.to_csv('행정동_기준_동별데이터_1031.csv')

In [137]:
result_df2['DPTM_NUM'].describe()

count    426.000000
mean       0.063380
std        0.207678
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.500000
Name: DPTM_NUM, dtype: float64

### 버스 데이터 추가

In [130]:
copy_df = result_df2.copy()
copy_df['GU_DONG'] = copy_df['GU'] + copy_df['DONG']
copy_df.head()

,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,...,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,GU,DONG,DONG_CODE,GU_DONG
2,18.0,0.0,1.0,2.0,2.0,3.0,56.666667,2.0,5.666667,1.0,...,0.0,4.0,18816.0,7.333333,39.666667,1.666667,종로구,청운효자동,1111051500,종로구청운효자동
12,19.0,10.0,0.0,2.0,0.0,0.0,41.666667,3.0,11.666667,5.0,...,0.0,20.0,22848.0,10.333333,100.666667,2.666667,종로구,사직동,1111053000,종로구사직동
24,2.0,10.0,0.0,0.0,1.0,1.0,22.000000,1.0,7.000000,1.0,...,0.0,2.0,14112.0,1.000000,52.000000,1.000000,종로구,삼청동,1111054000,종로구삼청동
31,24.0,0.0,1.0,4.0,1.0,1.0,41.000000,1.0,5.000000,5.0,...,0.0,8.0,6048.0,5.000000,30.000000,1.000000,종로구,부암동,1111055000,종로구부암동
34,28.0,0.0,0.0,0.0,0.0,1.0,21.000000,1.0,7.000000,0.0,...,0.0,6.0,4032.0,10.000000,36.000000,0.000000,종로구,평창동,1111056000,종로구평창동


In [131]:
bus_df = pd.read_csv('동별_버스정보.csv')
bus_df

,GU_DONG,BSTP_KIND_TPCD
0,강남구개포1동,13
1,강남구개포2동,20
2,강남구개포4동,19
3,강남구개포동,20
4,강남구논현1동,7
...,...,...
678,중랑구신내2동,9
679,중랑구신내동,16
680,중랑구중화1동,7
681,중랑구중화2동,13


In [132]:
merge_df = pd.merge(copy_df, bus_df)
set(copy_df['GU_DONG'].unique()) - set(merge_df['GU_DONG'].unique())

{'강동구상일1동', '강동구상일2동', '강동구암사2동', '구로구항동', '노원구상계8동'}

In [133]:
bus_df = pd.read_csv('동별_버스정보_수정.csv')
merge_df = pd.merge(copy_df, bus_df)
merge_df.drop('GU_DONG',axis=1,inplace=True)

In [134]:
merge_df.to_csv('행정동_기준_동별데이터_버스추가_1031.csv')

In [135]:
merge_df

,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,...,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,GU,DONG,DONG_CODE,BUS_CNT
0,18.000000,0.000000,1.0,2.000000,2.000000,3.0,56.666667,2.000000,5.666667,1.000000,...,0.0,4.000000,18816.000000,7.333333,39.666667,1.666667,종로구,청운효자동,1111051500,14.0
1,19.000000,10.000000,0.0,2.000000,0.000000,0.0,41.666667,3.000000,11.666667,5.000000,...,0.0,20.000000,22848.000000,10.333333,100.666667,2.666667,종로구,사직동,1111053000,16.0
2,2.000000,10.000000,0.0,0.000000,1.000000,1.0,22.000000,1.000000,7.000000,1.000000,...,0.0,2.000000,14112.000000,1.000000,52.000000,1.000000,종로구,삼청동,1111054000,11.0
3,24.000000,0.000000,1.0,4.000000,1.000000,1.0,41.000000,1.000000,5.000000,5.000000,...,0.0,8.000000,6048.000000,5.000000,30.000000,1.000000,종로구,부암동,1111055000,9.0
4,28.000000,0.000000,0.0,0.000000,0.000000,1.0,21.000000,1.000000,7.000000,0.000000,...,0.0,6.000000,4032.000000,10.000000,36.000000,0.000000,종로구,평창동,1111056000,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,...,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000,강동구,성내2동,1174065000,11.0
422,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,...,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000,강동구,성내3동,1174066000,9.0
423,100.000000,20.000000,1.0,6.000000,1.000000,0.0,119.000000,0.000000,9.000000,10.000000,...,0.0,76.000000,4327.000000,10.000000,63.000000,1.000000,강동구,길동,1174068500,28.0
424,38.000000,0.000000,0.0,2.000000,1.500000,1.0,32.500000,1.000000,5.500000,2.000000,...,0.5,17.000000,2163.500000,0.000000,17.500000,1.000000,강동구,둔촌1동,1174069000,4.0


In [136]:
merge_df['DPTM_NUM'].describecribe()

count    426.000000
mean       0.063380
std        0.207678
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.500000
Name: DPTM_NUM, dtype: float64